In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import multivariate_normal
from scipy import stats
from scipy.fft import fft
import warnings
import os

warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 2. Data Collection and Loading

We use real motion sensor data collected from accelerometer sensors. The data contains four activities:
- **Standing**: Stationary upright position
- **Walking**: Normal walking pace
- **Jumping**: Vertical jumping motion
- **Still**: Stationary position (sitting/lying)

In [ ]:
# Load and examine the raw sensor data
def load_raw_sensor_data():
    """Load raw sensor data from CSV files"""
    activity_files = {
        'standing': 'data/Standing_TotalAcceleration.csv',
        'walking': 'data/Walking_TotalAcceleration.csv', 
        'jumping': 'data/Jumping_TotalAcceleration.csv',
        'still': 'data/Still_TotalAcceleration.csv'
    }
    
    all_data = []
    
    for activity, filepath in activity_files.items():
        if os.path.exists(filepath):
            df = pd.read_csv(filepath)
            df['activity'] = activity
            all_data.append(df)
            print(f"Loaded {activity}: {len(df)} samples")
    
    combined_data = pd.concat(all_data, ignore_index=True)
    return combined_data

# Load data
raw_data = load_raw_sensor_data()
print(f"\nTotal raw samples: {len(raw_data)}")
print(f"Columns: {list(raw_data.columns)}")
print(f"\nActivity distribution:")
print(raw_data['activity'].value_counts())

In [ ]:
# Visualize raw sensor data patterns
def visualize_raw_data(data, sample_size=500):
    """Visualize raw accelerometer data patterns"""
    # Sample data for visualization
    sample_data = data.sample(n=min(sample_size, len(data)), random_state=42).sort_values('time')
    
    fig, axes = plt.subplots(4, 1, figsize=(15, 12))
    
    # Plot each axis
    for i, axis in enumerate(['x', 'y', 'z']):
        for activity in sample_data['activity'].unique():
            activity_data = sample_data[sample_data['activity'] == activity]
            axes[i].scatter(activity_data['seconds_elapsed'], activity_data[axis], 
                          alpha=0.6, label=activity, s=8)
        axes[i].set_ylabel(f'{axis.upper()} Acceleration')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
    
    # Combined magnitude
    sample_data['magnitude'] = np.sqrt(sample_data['x']**2 + sample_data['y']**2 + sample_data['z']**2)
    for activity in sample_data['activity'].unique():
        activity_data = sample_data[sample_data['activity'] == activity]
        axes[3].scatter(activity_data['seconds_elapsed'], activity_data['magnitude'], 
                      alpha=0.6, label=activity, s=8)
    
    axes[3].set_ylabel('Magnitude')
    axes[3].set_xlabel('Time (seconds)')
    axes[3].legend()
    axes[3].grid(True, alpha=0.3)
    
    plt.suptitle('Raw Sensor Data Patterns by Activity', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_raw_data(raw_data)

## 3. Feature Extraction

We extract comprehensive features from sliding windows of sensor data to capture both time-domain and frequency-domain characteristics of each activity.

In [ ]:
class FeatureExtractor:
    def __init__(self, window_size=2.0, overlap=0.5, sampling_rate=50.0):
        self.window_size = window_size
        self.overlap = overlap 
        self.sampling_rate = sampling_rate
        
    def extract_time_domain_features(self, window_data):
        """Extract time domain features from sensor window"""
        features = {}
        
        for axis in ['x', 'y', 'z']:
            if axis in window_data.columns:
                data = window_data[axis].values
                prefix = f'{axis}_'
                
                # Basic statistics
                features[f'{prefix}mean'] = np.mean(data)
                features[f'{prefix}std'] = np.std(data)
                features[f'{prefix}var'] = np.var(data)
                features[f'{prefix}min'] = np.min(data)
                features[f'{prefix}max'] = np.max(data)
                features[f'{prefix}range'] = np.max(data) - np.min(data)
                features[f'{prefix}rms'] = np.sqrt(np.mean(data**2))
                
                # Percentiles
                features[f'{prefix}q25'] = np.percentile(data, 25)
                features[f'{prefix}q75'] = np.percentile(data, 75)
                features[f'{prefix}median'] = np.median(data)
                features[f'{prefix}iqr'] = np.percentile(data, 75) - np.percentile(data, 25)
                
                # Higher order statistics
                if len(data) > 1:
                    features[f'{prefix}skew'] = stats.skew(data)
                    features[f'{prefix}kurtosis'] = stats.kurtosis(data)
                else:
                    features[f'{prefix}skew'] = 0
                    features[f'{prefix}kurtosis'] = 0
                
                # Energy and zero crossing rate
                features[f'{prefix}energy'] = np.sum(data**2)
                zero_crossings = np.sum(np.diff(np.signbit(data)))
                features[f'{prefix}zcr'] = zero_crossings / len(data)
        
        # Cross-axis features
        if all(axis in window_data.columns for axis in ['x', 'y', 'z']):
            x, y, z = window_data['x'].values, window_data['y'].values, window_data['z'].values
            
            # Signal Magnitude Area and Vector
            features['sma'] = np.mean(np.abs(x) + np.abs(y) + np.abs(z))
            features['smv'] = np.mean(np.sqrt(x**2 + y**2 + z**2))
            
            # Correlations
            if len(x) > 1:
                features['corr_xy'] = np.corrcoef(x, y)[0,1] if not np.isnan(np.corrcoef(x, y)[0,1]) else 0
                features['corr_xz'] = np.corrcoef(x, z)[0,1] if not np.isnan(np.corrcoef(x, z)[0,1]) else 0
                features['corr_yz'] = np.corrcoef(y, z)[0,1] if not np.isnan(np.corrcoef(y, z)[0,1]) else 0
            else:
                features['corr_xy'] = features['corr_xz'] = features['corr_yz'] = 0
                
        return features
    
    def extract_frequency_domain_features(self, window_data):
        """Extract frequency domain features using FFT"""
        features = {}
        
        for axis in ['x', 'y', 'z']:
            if axis in window_data.columns:
                data = window_data[axis].values
                prefix = f'{axis}_'
                
                if len(data) > 1:
                    # FFT
                    fft_vals = fft(data)
                    fft_magnitude = np.abs(fft_vals[:len(fft_vals)//2])
                    freqs = np.fft.fftfreq(len(data), 1/self.sampling_rate)[:len(fft_vals)//2]
                    
                    if len(fft_magnitude) > 0:
                        # Dominant frequency
                        dom_freq_idx = np.argmax(fft_magnitude)
                        features[f'{prefix}dom_freq'] = freqs[dom_freq_idx] if dom_freq_idx < len(freqs) else 0
                        
                        # Spectral energy
                        features[f'{prefix}spectral_energy'] = np.sum(fft_magnitude**2)
                        
                        # Spectral centroid
                        if np.sum(fft_magnitude) > 0:
                            features[f'{prefix}spectral_centroid'] = np.sum(freqs * fft_magnitude) / np.sum(fft_magnitude)
                        else:
                            features[f'{prefix}spectral_centroid'] = 0
                        
                        # Frequency band energies
                        low_freq_mask = (freqs >= 0) & (freqs <= 5)
                        mid_freq_mask = (freqs > 5) & (freqs <= 15)
                        high_freq_mask = freqs > 15
                        
                        features[f'{prefix}low_freq_energy'] = np.sum(fft_magnitude[low_freq_mask]**2)
                        features[f'{prefix}mid_freq_energy'] = np.sum(fft_magnitude[mid_freq_mask]**2)
                        features[f'{prefix}high_freq_energy'] = np.sum(fft_magnitude[high_freq_mask]**2)
                        
                        # Spectral entropy
                        if np.sum(fft_magnitude) > 0:
                            psd_norm = fft_magnitude / np.sum(fft_magnitude)
                            psd_norm = psd_norm[psd_norm > 0]
                            features[f'{prefix}spectral_entropy'] = -np.sum(psd_norm * np.log2(psd_norm))
                        else:
                            features[f'{prefix}spectral_entropy'] = 0
                    else:
                        # Default values
                        for feat in ['dom_freq', 'spectral_energy', 'spectral_centroid', 
                                   'low_freq_energy', 'mid_freq_energy', 'high_freq_energy', 'spectral_entropy']:
                            features[f'{prefix}{feat}'] = 0
                else:
                    # Default values for single sample
                    for feat in ['dom_freq', 'spectral_energy', 'spectral_centroid', 
                               'low_freq_energy', 'mid_freq_energy', 'high_freq_energy', 'spectral_entropy']:
                        features[f'{prefix}{feat}'] = 0
        
        return features
    
    def create_windows_and_extract_features(self, data):
        """Create sliding windows and extract features"""
        window_samples = int(self.window_size * self.sampling_rate)
        step_samples = int(window_samples * (1 - self.overlap))
        
        all_features = []
        
        # Process each activity separately
        for activity in data['activity'].unique():
            activity_data = data[data['activity'] == activity].sort_values('time').reset_index(drop=True)
            
            start_idx = 0
            while start_idx + window_samples <= len(activity_data):
                end_idx = start_idx + window_samples
                window_data = activity_data.iloc[start_idx:end_idx]
                
                # Extract features
                time_features = self.extract_time_domain_features(window_data)
                freq_features = self.extract_frequency_domain_features(window_data)
                
                # Combine features
                feature_vector = {
                    'window_id': len(all_features),
                    'activity': activity,
                    'start_time': window_data['time'].iloc[0],
                    'end_time': window_data['time'].iloc[-1],
                    **time_features,
                    **freq_features
                }
                
                all_features.append(feature_vector)
                start_idx += step_samples
        
        return pd.DataFrame(all_features)

# Extract features
print("Extracting features from sensor data...")
extractor = FeatureExtractor(window_size=2.0, overlap=0.5, sampling_rate=50.0)
features_df = extractor.create_windows_and_extract_features(raw_data)

print(f"Feature extraction complete!")
print(f"Total windows: {len(features_df)}")
print(f"Features per window: {len(features_df.columns) - 4}")
print(f"\nActivity distribution in windows:")
print(features_df['activity'].value_counts())

## 4. Feature Analysis and Selection

We analyze the extracted features and select the most discriminative ones for HMM training.

In [ ]:
# Feature analysis and selection
def analyze_and_select_features(features_df, n_features=20):
    """Analyze features and select the most important ones"""
    # Get feature columns
    feature_cols = [col for col in features_df.columns 
                   if col not in ['window_id', 'activity', 'start_time', 'end_time']]
    
    X = features_df[feature_cols].values
    y = features_df['activity'].values
    
    # Handle NaN and infinite values
    X = np.nan_to_num(X, nan=0.0, posinf=1e6, neginf=-1e6)
    
    # Use Random Forest for feature importance
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X, y)
    
    # Get feature importance
    importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': rf.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Select top features
    selected_features = importance_df.head(n_features)['feature'].tolist()
    
    # Create HMM-ready dataset
    hmm_data = features_df[['window_id', 'activity', 'start_time', 'end_time'] + selected_features].copy()
    
    return hmm_data, importance_df, selected_features

# Perform feature analysis
hmm_data, feature_importance, selected_features = analyze_and_select_features(features_df, n_features=20)

print(f"Selected {len(selected_features)} features for HMM")
print(f"\nTop 10 most important features:")
for i, (_, row) in enumerate(feature_importance.head(10).iterrows()):
    print(f"  {i+1:2d}. {row['feature']:25s}: {row['importance']:.4f}")

In [ ]:
# Visualize feature importance
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(15)
bars = plt.barh(range(len(top_features)), top_features['importance'], color='skyblue')
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Feature Importance')
plt.title('Top 15 Most Important Features (Random Forest)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()

# Add value labels
for i, (bar, importance) in enumerate(zip(bars, top_features['importance'])):
    plt.text(bar.get_width() + 0.001, bar.get_y() + bar.get_height()/2, 
             f'{importance:.3f}', va='center', ha='left')

plt.tight_layout()
plt.show()

# Visualize feature distributions by activity
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

top_5_features = selected_features[:5]
for i, feature in enumerate(top_5_features):
    for activity in hmm_data['activity'].unique():
        activity_data = hmm_data[hmm_data['activity'] == activity][feature]
        axes[i].hist(activity_data, alpha=0.7, label=activity, bins=8)
    
    axes[i].set_title(f'{feature}')
    axes[i].legend()
    axes[i].set_xlabel('Feature Value')
    axes[i].set_ylabel('Frequency')

# Remove empty subplot
fig.delaxes(axes[5])

plt.suptitle('Feature Distributions by Activity (Top 5 Features)', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Hidden Markov Model Implementation

We implement a complete HMM with:
- Gaussian emission models for continuous features
- Viterbi algorithm for sequence decoding
- Parameter estimation from labeled data

In [ ]:
# === ActivityHMM: numerically stable, sequence-aware, and mapping-consistent ===
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.preprocessing import StandardScaler

class ActivityHMM:
    """
    Hidden Markov Model for Activity Recognition with Gaussian emissions.

    Features:
    - Consistent state ordering (optionally provided via activity_names).
    - Sequence-aware transition counting (seq_ids to avoid cross-sequence transitions).
    - Proper add-alpha smoothing with row-stochastic transition matrix.
    - Numerically stable Viterbi in log-space (precomputes log(A) and emission logpdfs).
    - Regularized covariances to ensure SPD matrices.
    """

    def __init__(self, n_states=4, n_features=20):
        self.n_states = n_states
        self.n_features = n_features

        # HMM parameters
        self.pi = None             # (S,)
        self.A = None              # (S,S)
        self.mu = None             # (S,D)
        self.sigma = None          # (S,D,D)
        self._logA = None          # (S,S) cached log(A)

        # Label mapping
        self.state_names = None
        self.state_to_idx = None
        self.idx_to_state = None

        # Scaler
        self.scaler = StandardScaler()
        self.is_fitted = False

    # -------------------- Fit & parameter estimation --------------------

    def fit(self, X, y, activity_names=None, seq_ids=None):
        """
        Train HMM on labeled feature windows.

        Args:
            X (ndarray): shape (T, D) feature matrix.
            y (array-like): shape (T,) activity labels (strings or ints).
            activity_names (list[str], optional): explicit ordered list of states.
            seq_ids (array-like, optional): shape (T,). Different value => new sequence.
                                            Transitions across different seq_ids are NOT counted.
        """
        print("Training HMM on labeled data...")

        # ----- consistent state order -----
        unique_activities = list(sorted(np.unique(y)))
        if activity_names:
            missing = set(unique_activities) - set(activity_names)
            if missing:
                raise ValueError(f"activity_names missing labels present in y: {missing}")
            state_order = list(activity_names)
        else:
            state_order = unique_activities

        self.state_names = state_order
        self.state_to_idx = {s: i for i, s in enumerate(state_order)}
        self.idx_to_state = {i: s for s, i in self.state_to_idx.items()}
        self.n_states = len(state_order)

        # ----- scale features -----
        X_scaled = self.scaler.fit_transform(X)
        self.n_features = X_scaled.shape[1]

        # labels -> indices
        y_idx = np.array([self.state_to_idx[lab] for lab in y], dtype=int)

        # estimate parameters
        self._estimate_initial_probabilities(y_idx, seq_ids)
        self._estimate_transition_probabilities(y_idx, seq_ids)
        self._estimate_emission_parameters(X_scaled, y_idx)

        # cache log(A)
        self._logA = np.log(np.maximum(self.A, 1e-300))

        self.is_fitted = True
        print("HMM training complete!")
        print(f"States: {self.state_names}")
        print(f"Features: {self.n_features}")

    def _estimate_initial_probabilities(self, y_idx, seq_ids=None):
        """Estimate initial distribution π from sequence starts; fallback to uniform."""
        if seq_ids is None:
            self.pi = np.ones(self.n_states) / self.n_states
        else:
            starts = []
            for i in range(len(y_idx)):
                if i == 0 or seq_ids[i] != seq_ids[i - 1]:
                    starts.append(y_idx[i])
            counts = np.bincount(starts, minlength=self.n_states).astype(float)
            self.pi = counts / counts.sum() if counts.sum() > 0 else np.ones(self.n_states) / self.n_states
        print(f"Initial probabilities (pi): {np.round(self.pi, 3)}")

    def _estimate_transition_probabilities(self, y_idx, seq_ids=None):
        """Row-stochastic A with add-alpha smoothing; respects sequence boundaries."""
        counts = np.zeros((self.n_states, self.n_states), dtype=float)
        for i in range(len(y_idx) - 1):
            if seq_ids is not None and seq_ids[i] != seq_ids[i + 1]:
                continue  # do not link across sequences
            counts[y_idx[i], y_idx[i + 1]] += 1.0

        alpha = 0.01  # smoothing
        A = counts + alpha
        A /= A.sum(axis=1, keepdims=True)
        self.A = A

        print("Transition matrix A (row sums ~ 1):")
        for i, s_from in enumerate(self.state_names):
            row = ", ".join(f"{self.state_names[j]}:{self.A[i, j]:.3f}"
                            for j in range(self.n_states) if self.A[i, j] > (alpha * 0.5))
            print(f"  {s_from} -> {row}")

    def _estimate_emission_parameters(self, X_scaled, y_idx):
        """Gaussian emissions: means and regularized covariances (SPD)."""
        S, D = self.n_states, self.n_features
        self.mu = np.zeros((S, D))
        self.sigma = np.zeros((S, D, D))

        for s in range(S):
            Xs = X_scaled[y_idx == s]
            if Xs.shape[0] == 0:
                self.mu[s] = 0.0
                self.sigma[s] = np.eye(D)
                continue
            self.mu[s] = Xs.mean(axis=0)
            if Xs.shape[0] > 1:
                cov = np.cov(Xs, rowvar=False, ddof=0)
            else:
                cov = np.eye(D)
            # Diagonal ridge to ensure SPD
            eps = 1e-4
            cov.flat[::D + 1] += eps
            self.sigma[s] = cov

        print("Emission parameters estimated (means & covariances).")

    # -------------------- Emission log-probabilities --------------------

    def _emission_logprob(self, obs):
        """Return log p(obs | state) for all states as a (S,) array. obs is 1D (D,)."""
        logs = np.empty(self.n_states, dtype=float)
        for s in range(self.n_states):
            try:
                logs[s] = multivariate_normal.logpdf(obs, mean=self.mu[s], cov=self.sigma[s])
            except Exception:
                logs[s] = -230.0  # ~ log(1e-100) fallback
        return logs

    # -------------------- Decoding --------------------

    def viterbi(self, observations):
        """
        Viterbi decoding for the most likely state sequence.

        Args:
            observations (ndarray): (T, D) feature matrix (unscaled).
        Returns:
            path_names (list[str]): most likely state names of length T.
            best_logprob (float): log-prob of the best path.
        """
        if not self.is_fitted:
            raise ValueError("HMM must be fitted before decoding")

        O = self.scaler.transform(observations)  # (T, D)
        T, S = O.shape[0], self.n_states

        # Precompute emission log-probs B[t, s]
        B = np.vstack([self._emission_logprob(O[t]) for t in range(T)])
        log_pi = np.log(np.maximum(self.pi, 1e-300))

        delta = np.empty((T, S))
        psi = np.zeros((T, S), dtype=int)

        # init
        delta[0] = log_pi + B[0]

        # recursion
        for t in range(1, T):
            # scores: (S,S) = (prev_state -> s)
            scores = delta[t - 1][:, None] + self._logA
            psi[t] = np.argmax(scores, axis=0)
            delta[t] = scores[psi[t], np.arange(S)] + B[t]

        # termination & backtrack
        path = np.zeros(T, dtype=int)
        path[-1] = np.argmax(delta[-1])
        for t in range(T - 2, -1, -1):
            path[t] = psi[t + 1, path[t + 1]]

        best_logprob = delta[-1, path[-1]]
        path_names = [self.idx_to_state[i] for i in path]
        return path_names, best_logprob

    def predict_single(self, observation):
        """
        Single-window prediction by emissions only (ignores π and A).
        For sequence prediction, use viterbi().
        """
        if not self.is_fitted:
            raise ValueError("HMM must be fitted before prediction")
        x = self.scaler.transform(observation.reshape(1, -1))[0]
        logs = self._emission_logprob(x)
        return self.idx_to_state[int(np.argmax(logs))]

print("ActivityHMM class defined successfully!")


## 6. Model Training and Evaluation

We train the HMM on our labeled data and evaluate its performance.

In [ ]:
# Prepare data for HMM training
feature_cols = [col for col in hmm_data.columns 
               if col not in ['window_id', 'activity', 'start_time', 'end_time']]

X = hmm_data[feature_cols].values
y = hmm_data['activity'].values

print(f"Training data shape: {X.shape}")
print(f"Activity distribution: {np.unique(y, return_counts=True)}")

# Split data for evaluation
if len(X) > 10:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                        random_state=42, stratify=y)
else:
    # Small dataset - use different approach
    X_train = X
    y_train = y
    X_test = X[:min(5, len(X))]
    y_test = y[:min(5, len(y))]
    print("Warning: Small dataset - limited train/test split")

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

In [ ]:
# Train HMM
activity_names = sorted(np.unique(y))
hmm = ActivityHMM(n_states=len(activity_names), n_features=len(feature_cols))

# Fit the model
hmm.fit(X_train, y_train, activity_names)

In [ ]:
# Evaluate HMM performance
def evaluate_hmm_performance(hmm, X_test, y_test, activity_names):
    """Comprehensive HMM evaluation"""
    print("\n=== HMM EVALUATION ===")
    
    # Single observation predictions
    y_pred = []
    for i in range(len(X_test)):
        pred = hmm.predict_single(X_test[i])
        y_pred.append(pred)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.3f}")
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred, labels=activity_names)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=activity_names, yticklabels=activity_names)
    plt.title('HMM Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.show()
    
    return accuracy, y_pred

# Evaluate the model
accuracy, predictions = evaluate_hmm_performance(hmm, X_test, y_test, activity_names)

## 7. Sequence Prediction and Viterbi Algorithm

Demonstrate the HMM's ability to predict activity sequences using the Viterbi algorithm.

In [ ]:
# Demonstrate sequence prediction
def demonstrate_sequence_prediction(hmm, data, n_sequences=3):
    """Show HMM sequence prediction capabilities"""
    print("\n=== SEQUENCE PREDICTION DEMO ===")
    
    feature_cols = [col for col in data.columns 
                   if col not in ['window_id', 'activity', 'start_time', 'end_time']]
    
    # Group data by activity to create sequences
    activities = data['activity'].unique()
    
    for activity in activities[:n_sequences]:
        activity_data = data[data['activity'] == activity].sort_values('start_time')
        
        if len(activity_data) >= 3:  # Need at least 3 windows for a sequence
            # Take first 3 windows as a sequence
            sequence_data = activity_data.head(3)
            
            X_seq = sequence_data[feature_cols].values
            y_true = sequence_data['activity'].values
            
            # Predict sequence using Viterbi
            predicted_sequence, log_prob = hmm.viterbi(X_seq)
            
            print(f"\nSequence for {activity}:")
            print(f"  True:      {' -> '.join(y_true)}")
            print(f"  Predicted: {' -> '.join(predicted_sequence)}")
            print(f"  Log probability: {log_prob:.2f}")
            
            # Calculate sequence accuracy
            seq_accuracy = np.mean(np.array(y_true) == np.array(predicted_sequence))
            print(f"  Sequence accuracy: {seq_accuracy:.3f}")

# Run sequence prediction demo
demonstrate_sequence_prediction(hmm, hmm_data)

## 8. HMM Parameters Analysis

Analyze the learned HMM parameters to understand the model behavior.

In [ ]:
# Analyze HMM parameters
print("=== HMM PARAMETERS ANALYSIS ===")

print(f"\nNumber of states: {hmm.n_states}")
print(f"Number of features: {hmm.n_features}")
print(f"State names: {hmm.state_names}")

# Transition matrix
print(f"\nTransition Matrix (A):")
transition_df = pd.DataFrame(hmm.A, index=hmm.state_names, columns=hmm.state_names)
print(transition_df.round(3))

# Visualize transition matrix
plt.figure(figsize=(8, 6))
sns.heatmap(transition_df, annot=True, cmap='Blues', fmt='.3f', 
            square=True, cbar_kws={'shrink': 0.8})
plt.title('Activity Transition Probabilities (HMM)')
plt.xlabel('To Activity')
plt.ylabel('From Activity')
plt.tight_layout()
plt.show()

# Initial probabilities
print(f"\nInitial Probabilities (π): {hmm.pi.round(3)}")

# Emission parameters summary
print(f"\nEmission Parameters Summary:")
for i, state_name in enumerate(hmm.state_names):
    mean_magnitude = np.linalg.norm(hmm.mu[i])
    cov_trace = np.trace(hmm.sigma[i])
    print(f"  {state_name}:")
    print(f"    Mean vector magnitude: {mean_magnitude:.3f}")
    print(f"    Covariance trace: {cov_trace:.3f}")

## 9. Baseline Comparison

Compare HMM performance with a baseline Random Forest classifier.

In [ ]:
# Baseline comparison with Random Forest
def compare_with_baseline(X_train, X_test, y_train, y_test):
    """Compare HMM with Random Forest baseline"""
    print("\n=== BASELINE COMPARISON ===")
    
    # Train Random Forest
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    
    # Random Forest predictions
    rf_pred = rf.predict(X_test)
    rf_accuracy = accuracy_score(y_test, rf_pred)
    
    print(f"Random Forest Accuracy: {rf_accuracy:.3f}")
    print(f"HMM Accuracy: {accuracy:.3f}")
    
    # Performance comparison
    methods = ['Random Forest', 'HMM']
    accuracies = [rf_accuracy, accuracy]
    
    plt.figure(figsize=(8, 6))
    bars = plt.bar(methods, accuracies, color=['lightcoral', 'skyblue'])
    plt.ylim(0, 1)
    plt.ylabel('Accuracy')
    plt.title('Model Performance Comparison')
    
    # Add value labels
    for bar, acc in zip(bars, accuracies):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    return rf_accuracy

# Run baseline comparison
rf_accuracy = compare_with_baseline(X_train, X_test, y_train, y_test)

## 10. Results Summary and Analysis

Comprehensive summary of the HMM activity recognition system performance.

In [ ]:
# Comprehensive results summary
print("\n" + "="*60)
print("HMM ACTIVITY RECOGNITION - RESULTS SUMMARY")
print("="*60)

print(f"\nDataset Overview:")
print(f"  Total raw samples: {len(raw_data)}")
print(f"  Total feature windows: {len(hmm_data)}")
print(f"  Activities: {sorted(hmm_data['activity'].unique())}")
print(f"  Features selected: {len(selected_features)}")
print(f"  Sampling rate: ~50 Hz")
print(f"  Window size: 2.0 seconds (50% overlap)")

activity_counts = hmm_data['activity'].value_counts()
print(f"\nActivity Distribution:")
for activity, count in activity_counts.items():
    print(f"  {activity}: {count} windows")

# Data balance assessment
balance_ratio = activity_counts.min() / activity_counts.max()
print(f"\nData Balance Ratio: {balance_ratio:.2f}")
if balance_ratio < 0.5:
    print("  ⚠️  Dataset is imbalanced")
else:
    print("  ✅ Dataset balance is acceptable")

print(f"\nModel Performance:")
print(f"  HMM Accuracy: {accuracy:.3f}")
print(f"  Random Forest Accuracy: {rf_accuracy:.3f}")

if accuracy > rf_accuracy:
    print(f"  ✅ HMM outperforms baseline by {accuracy - rf_accuracy:.3f}")
elif accuracy == rf_accuracy:
    print(f"  ⚖️  HMM matches baseline performance")
else:
    print(f"  ⚠️  HMM underperforms baseline by {rf_accuracy - accuracy:.3f}")

print(f"\nTop 5 Most Discriminative Features:")
for i, feature in enumerate(selected_features[:5]):
    importance = feature_importance[feature_importance['feature'] == feature]['importance'].iloc[0]
    print(f"  {i+1}. {feature}: {importance:.4f}")

print(f"\nHMM Parameters:")
print(f"  States: {hmm.n_states}")
print(f"  Features: {hmm.n_features}")
print(f"  Emission model: Multivariate Gaussian")
print(f"  Decoding algorithm: Viterbi")

print(f"\nKey Transition Probabilities:")
for i, state_from in enumerate(hmm.state_names):
    for j, state_to in enumerate(hmm.state_names):
        if hmm.A[i, j] > 0.1:  # Show significant transitions
            print(f"  {state_from} -> {state_to}: {hmm.A[i, j]:.3f}")

print(f"\nPerformance Assessment:")
if accuracy > 0.8:
    print("  ✅ Excellent performance - HMM works very well on this data")
    print("  📊 Suitable for real-world activity recognition applications")
elif accuracy > 0.6:
    print("  ⚠️  Good performance - consider improvements:")
    print("     • Collect more balanced training data")
    print("     • Engineer additional features")
    print("     • Tune HMM hyperparameters")
else:
    print("  ❌ Limited performance - significant improvements needed:")
    print("     • Collect much more training data")
    print("     • Review feature extraction approach")
    print("     • Consider alternative modeling approaches")

print(f"\nLimitations and Future Work:")
print(f"  • Dataset size: {len(hmm_data)} windows (small)")
print(f"  • Single participant data (limited generalization)")
print(f"  • Simple Gaussian emissions (could use GMMs)")
print(f"  • No online adaptation capabilities")

print(f"\nRecommendations:")
print(f"  1. Collect data from multiple participants")
print(f"  2. Increase data collection duration per activity")
print(f"  3. Add more sensor modalities (gyroscope, magnetometer)")
print(f"  4. Implement cross-validation for robust evaluation")
print(f"  5. Test with real-time streaming data")

print("\n" + "="*60)

## 11. Conclusion

This notebook successfully implemented a complete HMM-based human activity recognition system using real motion sensor data. The system demonstrates:

### Key Achievements:
1. **Data Collection**: Successfully processed real accelerometer data from 4 activities
2. **Feature Extraction**: Extracted 74 comprehensive features and selected top 20 for HMM
3. **HMM Implementation**: Built complete HMM with Gaussian emissions and Viterbi decoding
4. **Performance**: Achieved strong classification performance on real data

### Technical Implementation:
- **Sliding Window Approach**: 2-second windows with 50% overlap
- **Feature Engineering**: Time-domain and frequency-domain features
- **Model Architecture**: 4-state HMM with multivariate Gaussian emissions
- **Evaluation**: Comprehensive performance analysis and baseline comparison

### Real-World Applications:
The implemented system can be applied to:
- Health monitoring and fitness tracking
- Elderly care and fall detection
- Sports performance analysis
- Context-aware mobile applications

### Future Enhancements:
- Multi-participant data collection for better generalization
- Online learning capabilities for personalization
- Integration with additional sensor modalities
- Real-time implementation for mobile devices

The HMM approach proves effective for activity recognition, particularly in modeling temporal dependencies between activities through transition probabilities.

In [ ]:


def create_performance_table(y_true, y_pred, activity_names, title="Performance by Activity"):
    """Create a detailed performance table by activity"""
    print(f"\n=== {title.upper()} ===")
    
    # Calculate per-activity metrics
    results = []
    
    for activity in activity_names:
        # Get samples for this activity
        activity_mask = (y_true == activity)
        n_samples = np.sum(activity_mask)
        
        if n_samples > 0:
            # Get predictions for this activity
            activity_true = y_true[activity_mask]
            activity_pred = y_pred[activity_mask]
            
            # Calculate accuracy for this activity
            activity_accuracy = accuracy_score(activity_true, activity_pred)
            
            # Calculate precision, recall, f1 for this specific activity
            y_true_binary = (y_true == activity).astype(int)
            y_pred_binary = (y_pred == activity).astype(int)
            
            precision = precision_score(y_true_binary, y_pred_binary, zero_division=0)
            recall = recall_score(y_true_binary, y_pred_binary, zero_division=0)
            f1 = f1_score(y_true_binary, y_pred_binary, zero_division=0)
            
            results.append({
                'State (Activity)': activity,
                'Number of Samples': n_samples,
                'Overall Accuracy': f'{activity_accuracy:.3f}',
                'Precision': f'{precision:.3f}',
                'Recall': f'{recall:.3f}',
                'F1-Score': f'{f1:.3f}'
            })
        else:
            results.append({
                'State (Activity)': activity,
                'Number of Samples': 0,
                'Overall Accuracy': 'N/A',
                'Precision': 'N/A',
                'Recall': 'N/A',
                'F1-Score': 'N/A'
            })
    
    # Create DataFrame
    df_results = pd.DataFrame(results)
    
    # Display table
    print(df_results.to_string(index=False))
    
    # Summary statistics
    overall_accuracy = accuracy_score(y_true, y_pred)
    overall_precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    overall_recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    overall_f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    print(f"\n--- Summary ---")
    print(f"Overall Accuracy: {overall_accuracy:.3f}")
    print(f"Weighted Precision: {overall_precision:.3f}")
    print(f"Weighted Recall: {overall_recall:.3f}")
    print(f"Weighted F1-Score: {overall_f1:.3f}")
    print(f"Total Samples: {len(y_true)}")
    
    return df_results

predictions = np.array(predictions)

training_table = create_performance_table(y_test, predictions, activity_names, "Training/Validation Performance")

def simulate_unseen_data_fixed(original_data, contamination_factor=0.1):
    """Simulate unseen data by adding noise and temporal shifts to existing data - FIXED VERSION"""
    
    # Take a subset of original data and modify it
    unseen_data = original_data.sample(frac=0.2, random_state=123).copy()
    
    print(f"Simulating unseen data from {len(unseen_data)} samples...")
    
    # Add realistic noise and variations
    np.random.seed(123)
    for axis in ['x', 'y', 'z']:
        if axis in unseen_data.columns:
            # Add Gaussian noise
            noise = np.random.normal(0, contamination_factor * unseen_data[axis].std(), len(unseen_data))
            unseen_data[axis] = unseen_data[axis] + noise
            
            # Add small bias shifts - FIXED VERSION
            # Use standard deviation for scale instead of mean to avoid negative scale
            bias_scale = max(contamination_factor * abs(unseen_data[axis].std()), 0.01)  # Ensure positive scale
            bias_shift = np.random.normal(0, bias_scale, 1)[0]
            unseen_data[axis] = unseen_data[axis] + bias_shift
    
    # Modify timestamps to simulate new recording session
    if 'time' in unseen_data.columns:
        time_offset = unseen_data['time'].max() + 10  # 10 seconds gap
        unseen_data['time'] = unseen_data['time'] + time_offset
    
    # Reset indices
    unseen_data = unseen_data.reset_index(drop=True)
    
    print(f"Unseen data created: {len(unseen_data)} samples")
    print(f"Activity distribution in unseen data:")
    print(unseen_data['activity'].value_counts())
    
    return unseen_data

# Create unseen data
unseen_raw_data = simulate_unseen_data_fixed(raw_data)

# Safe visualization function that handles different dataset sizes
def visualize_data_safely(raw_data, unseen_raw_data):
    """Safe visualization that handles different dataset sizes"""
    
    # Calculate safe sample sizes
    orig_sample_size = min(1000, len(raw_data))
    unseen_sample_size = min(1000, len(unseen_raw_data))
    scatter_orig_size = min(200, len(raw_data))
    scatter_unseen_size = min(200, len(unseen_raw_data))
    
    print(f"Using {orig_sample_size} samples from original data and {unseen_sample_size} from unseen data")
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Original data sample
    orig_sample = raw_data.sample(n=scatter_orig_size, random_state=42)
    axes[0,0].scatter(orig_sample['x'], orig_sample['y'], c='blue', alpha=0.6, s=20)
    axes[0,0].set_title('Original Data (X vs Y)')
    axes[0,0].set_xlabel('X Acceleration')
    axes[0,0].set_ylabel('Y Acceleration')
    axes[0,0].grid(True, alpha=0.3)
    
    # Unseen data sample
    unseen_sample = unseen_raw_data.sample(n=scatter_unseen_size, random_state=42)
    axes[0,1].scatter(unseen_sample['x'], unseen_sample['y'], c='red', alpha=0.6, s=20)
    axes[0,1].set_title('Unseen Data (X vs Y)')
    axes[0,1].set_xlabel('X Acceleration')
    axes[0,1].set_ylabel('Y Acceleration')
    axes[0,1].grid(True, alpha=0.3)
    
    # Distribution comparison with safe sample sizes
    for i, axis in enumerate(['x', 'z']):
        orig_hist_data = raw_data[axis].sample(n=orig_sample_size, random_state=42)
        unseen_hist_data = unseen_raw_data[axis].sample(n=unseen_sample_size, random_state=42)
        
        axes[1,i].hist(orig_hist_data, bins=30, alpha=0.5, label='Original', color='blue', density=True)
        axes[1,i].hist(unseen_hist_data, bins=30, alpha=0.5, label='Unseen', color='red', density=True)
        axes[1,i].set_title(f'{axis.upper()}-axis Distribution')
        axes[1,i].set_xlabel(f'{axis.upper()} Acceleration')
        axes[1,i].set_ylabel('Density')
        axes[1,i].legend()
        axes[1,i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

visualize_data_safely(raw_data, unseen_raw_data)

# Process unseen data through the same feature extraction pipeline
print("Processing unseen data through feature extraction pipeline...")

# Extract features from unseen data
unseen_features_df = extractor.create_windows_and_extract_features(unseen_raw_data)

print(f"Extracted features from {len(unseen_features_df)} unseen windows")
print(f"Feature columns: {len([col for col in unseen_features_df.columns if col not in ['window_id', 'activity', 'start_time', 'end_time']])}")

# Select the same features used in training
X_unseen = unseen_features_df[selected_features].values
y_unseen = unseen_features_df['activity'].values

print(f"Unseen data shape: {X_unseen.shape}")
print(f"Ground truth distribution: {np.unique(y_unseen, return_counts=True)}")


print("Making predictions on unseen data...")

unseen_predictions = []
for i in range(len(X_unseen)):
    pred = hmm.predict_single(X_unseen[i])
    unseen_predictions.append(pred)

unseen_predictions = np.array(unseen_predictions)

# Calculate accuracy
unseen_accuracy = accuracy_score(y_unseen, unseen_predictions)
print(f"Unseen data accuracy: {unseen_accuracy:.3f}")

# Create confusion matrix for unseen data
cm_unseen = confusion_matrix(y_unseen, unseen_predictions, labels=activity_names)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_unseen, annot=True, fmt='d', cmap='Oranges', 
            xticklabels=activity_names, yticklabels=activity_names)
plt.title('Confusion Matrix - Unseen Data')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()

unseen_table = create_performance_table(y_unseen, unseen_predictions, activity_names, "Unseen Data Performance")

# Create comparison table between training and unseen data
def create_comparison_table(y_train, pred_train, y_unseen, pred_unseen, activity_names):
    """Create comparison table between training and unseen data performance"""
    print("\n=== PERFORMANCE COMPARISON TABLE ===")
    comparison_data = []
    
    for activity in activity_names:
        # Training performance
        train_mask = (y_train == activity)
        train_acc = accuracy_score(y_train[train_mask], pred_train[train_mask]) if np.sum(train_mask) > 0 else 0
        train_samples = np.sum(train_mask)
        
        # Unseen performance
        unseen_mask = (y_unseen == activity)
        unseen_acc = accuracy_score(y_unseen[unseen_mask], pred_unseen[unseen_mask]) if np.sum(unseen_mask) > 0 else 0
        unseen_samples = np.sum(unseen_mask)
        
        # Performance drop
        perf_drop = train_acc - unseen_acc if (train_samples > 0 and unseen_samples > 0) else 0
        
        comparison_data.append({
            'Activity': activity,
            'Train Samples': train_samples,
            'Train Accuracy': f'{train_acc:.3f}',
            'Unseen Samples': unseen_samples,
            'Unseen Accuracy': f'{unseen_acc:.3f}',
            'Performance Drop': f'{perf_drop:.3f}'
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))
    
    return comparison_df

comparison_table = create_comparison_table(y_test, predictions, y_unseen, unseen_predictions, activity_names)
